# DESaster Application Template
v. 042217

Features include Owner, Renter, Landlord, FinancialRecoveryProgram, TechnicalRecoveryProgram, Policies

## Required Modules

In [19]:
### External Modules
import sys, random, inspect
desaster_path = "/Users/geomando/Dropbox/github/SeaGrantSimulation"
sys.path.append(desaster_path)
import simpy
from simpy import Resource, Container, FilterStore
from simpy.util import start_delayed
import pandas as pd
import numpy as np
from scipy.stats import uniform, beta
import seaborn as sns
import matplotlib.pyplot as plt
import random

### DESaster Modules
from desaster.structures import *
from desaster.financial import *
from desaster.technical import *
from desaster.entities import *
from desaster.policies import *
from desaster.io import *

## Input Data
Input data must be MS Excel .xlsx file with format (column names and sheet names) of ../inputs/desaster_input_data_template.xlsx

In [20]:
scenario_file = '../inputs/desaster_input_data_template.xlsx'

__Create Pandas dataframe of attribute data for all owners to be modeled in the simulation.__

In [21]:
owners_df = pd.read_excel(scenario_file, sheetname='owners')
owners_df

,Name,Savings,Owner Insurance,Address,Cost,Occupancy,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,Alfred,5000,1,62 That St,1000,Mobile Home,1,1,1100,1920,100000,Slight
1,Bruce,250000,1,720 This Rd,4000,Single Family Dwelling,4,5,5000,1920,10000000,Extensive
2,Selena,2500,1,1001 Other Ave,1000,Single Family Dwelling,2,1,1200,1960,10000,Moderate
3,Fish,1000,1,26000 Out There Lane,2000,Single Family Dwelling,3,2,2000,2010,800000,Complete


__Create Pandas dataframe of attribute data for all vacant (no entity owners) homes for sale to be modeled in the simulation.__

In [22]:
sellers_df = pd.read_excel(scenario_file, sheetname='forsale_stock')
sellers_df

,Name,Savings,Owner Insurance,Address,Cost,Occupancy,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,Jerome,100000000,1,100 New Ave,1000,Mobile Home,1,1,1100,1920,100000,Slight
1,Barbara,100000000,1,101 New Ave,4000,Single Family Dwelling,4,5,5000,1920,10000000,Extensive
2,Lucas,100000000,1,102 New Ave,1000,Single Family Dwelling,2,1,1200,1960,10000,Moderate
3,Dick,100000000,1,103 New Ave,2000,Single Family Dwelling,3,2,2000,2010,800000,Complete


In [23]:
# forsale_stock_df = pd.read_excel(scenario_file, sheetname='forsale_stock')
# forsale_stock_df

__Create Pandas dataframe of attribute data for all renters to be modeled in the simulation.__

In [24]:
renters_df = pd.read_excel(scenario_file, sheetname='renters')
renters_df

,Name,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State,Landlord,Savings,Owner Insurance
0,Ivy,262 That St,Mobile Home,1000,1,1,700,1920,100000,Complete,Alice,10000,1
1,Edward,4720 This Rd,Single Family Dwelling,3000,3,2,5000,1920,10000000,Complete,Julie,2000,1
2,Oswald,2301 Other Ave,Single Family Dwelling,750,0,1,250,1960,10000,Complete,Gerry,50000,1
3,James,74000 Out There Lane,Single Family Dwelling,1500,2,2,2000,2010,800000,Complete,Sally,3000,1


__Create Pandas dataframe of attribute data for all vacant (no entity tenants) rentals to be modeled in the env.__

In [25]:
forrent_stock_df = pd.read_excel(scenario_file, sheetname='forrent_stock')
forrent_stock_df

,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,100 Old Ave,Mobile Home,100,1,1,700,1920,99999,Complete
1,101 Old Ave,Single Family Dwelling,100000,2,2,5000,1920,9999,Complete
2,102 Old Ave,Multi Family Dwelling,10,0,1,250,1960,9999,Complete
3,103 Old Ave,Single Family Dwelling,2000,3,2,2000,2010,800000,Complete


## Setup the Simulaiton Environment and Populate with Entities and Processes

__Set Simpy simulation environment__

In [26]:
env = simpy.Environment()

__Indicate whether the simulation will keep track of the stories of each entity in the simulation. This can also be set individually for each group of imported entities (e.g., each call to an entities.Entity class or subclass.__

In [27]:
write_story = True

__Define the probability distributions that can/will be used in the simulation to define various recovery program process event durations.__

In [28]:
# A determistic scalar
scalar_dist = DurationProbabilityDistribution(dist='scalar', loc=10) 
# A uniform distribution; min = loc, max = (loc + scale)
uni_dist = DurationProbabilityDistribution(dist='uniform', loc=5, scale=10) 
# A beta distribution; parameters define as numpy.stats.beta
beta_dist = DurationProbabilityDistribution(dist='beta', loc=5, scale=10, shape_a=2.0, shape_b=2.0)
# A weibull distribution; parameters define as numpy.stats.beta
wei_dist = DurationProbabilityDistribution(dist='weibull', loc=5, scale=10, shape_c=2.0)

__Instantiate the recovery program objects that will be used in the simulation. Each recovery program requires
specification of a duration probability distribution (set above). Unlike shown below, the distributions do not have to be the same for each program. Currently all are the same scalars to simplify interpretation and debugging.__

In [29]:
fema_prog = IndividualAssistance(env, staff=100, budget=100000, duration_prob_dist=scalar_dist, max_outlay=30000)
loan_prog = HomeLoan(env, staff=10, duration_prob_dist=scalar_dist)
insurance_prog = OwnersInsurance(env, staff=100, deductible=0.0, duration_prob_dist=scalar_dist)
inspection_prog = InspectionProgram(env, staff=1000, duration_prob_dist=scalar_dist)
assessment_prog = EngineeringAssessment(env, staff=1000, duration_prob_dist=scalar_dist)
permit_prog = PermitProgram(env, staff=1000, duration_prob_dist=scalar_dist) 
rebuild_prog = RepairProgram(env, staff=1000, materials=1000000000, duration_prob_dist=scalar_dist)

In [30]:
#inspect.getfullargspec(rebuild_prog.process).args # Useful to determine the arguments for the recovery programs

__Instantiate any recovery policies. Policies are custom classes that define different recovery program arrangements, sequences, logic, patience, etc. Currently only one is written Insurance_IA_Loan_Sequential, which very loosely mimics FEMA's required sequencing of funding requests. The policy also provides an example of how to model entity "patience" and process interuption.__

In [31]:
insurance_ia_loan = Insurance_IA_Loan_Sequential(env)
repair_stock = RepairBuildingStock(env, inspection_prog, assessment_prog, permit_prog, rebuild_prog)

__Create and populate Simpy FilterStores to use as vacant housing stocks. If input data includes an unsupported occupancy type, a warning will be given but simulation will attempt to proceed.__

In [32]:
forsale_stock = importSingleFamilyResidenceStock(env, forsale_stock_df)  # Owned housing stock
# forrent_stock = importSingleFamilyResidenceStock(env, forrent_stock_df)  # For rent housing stock

NameError: name 'forsale_stock_df' is not defined

__Create empty Simpy FilterStores to use as occupied housing stocks.__

In [ ]:
owned_stock = FilterStore(env)  # To put the residences associated with owners
rented_stock = FilterStore(env) # To put the residences associated with renters
# forsale_stock = FilterStore(env)

__Create a list of OwnerHousehold objects based on input data. Place each associated residence in the occupied housing stock.__

In [ ]:
owners = importOwnerHouseholds(env, owned_stock, owners_df, write_story)

__Create a list of RenterHousehold objects based on input data. Place each associated residence in the rented housing stock.__

In [ ]:
renters = importRenterHouseholds(env, rented_stock, renters_df, write_story)

In [ ]:
# sellers = importOwnerHouseholds(env, forsale_stock, forsale_stock_df)

__A basic master process that strings together recovery programs. This will work but is shown mainly as a simply illustration of how to make a master process to population the simulation with recovery programs and policies with different arrangements/logic.__

In [ ]:
# def seller_process(inspection_program, insurance_program, fema_program, loan_program, 
#                  assessment_program, permit_program, rebuild_program, forsale_stock, entity):
#     if entity.property.damage_state != 'None':
#         get_building = yield forsale_stock.get(entity.property)
#         get_building = yield forsale_stock.get(lambda getBuilding:
#                                                     getBuilding.address.lower() == entity.property.address.lower()
#                                             )
#         yield env.process(inspection_program.process(entity.property, entity))
#         yield env.process(insurance_program.process(entity))
#         yield env.process(fema_program.process(entity))
#         yield env.process(loan_program.process(entity))
#         yield env.process(assessment_program.process(entity.property, entity))
#         yield env.process(permit_program.process(entity.property, entity))
#         yield env.process(rebuild_program.process(entity.property, entity))
#         yield forsale_stock.put(get_building)

__A custom master process for landlords. Landlords are the owners and so are the ones to seek financial assistance for repairs. If repairs can't be made etc., evict the tenants.__

In [ ]:
def landlord_process(env, inspection_program, insurance_program, fema_program, loan_program, 
                 assessment_program, permit_program, rebuild_program, entity):
        
    yield env.process(inspection_program.process(entity.property, entity))
    
    if entity.property.damage_state != 'None':
        
        # If home is completely damaged, evict tenant
        if entity.property.damage_state == 'Complete':
            entity.tenant.prior_residence.append(entity.tenant.residence)
            entity.tenant.residence = None
            
            if entity.write_story == True:
                entity.tenant.story.append(
                '{0} was permanently evicted because the {1} was demolished. '.format(
                                                entity.tenant.name, entity.property.occupancy.lower()
                                                                                        )
                                            )

                entity.story.append(
                '{0} demolished their {1}. '.format(entity.name, entity.property.occupancy.lower())
                                    )
            return

        # Landlord search for financial assistance
        money_patience = 100000  # days until give up the search for rebuild money
        yield env.process(insurance_ia_loan.policy(insurance_prog, fema_prog, 
                                                          loan_prog, entity, money_patience
                                                         )
                                )

        if entity.gave_up_funding_search != None:
            entity.tenant.prior_residence.append(entity.tenant.residence)
            entity.tenant.residence = None
            
            if entity.write_story == True:
                entity.tenant.story.append(
                '{0} was permanently evicted because the {1} was not repaired. '.format(
                entity.tenant.name, entity.property.occupancy.lower())
                )
                
                entity.story.append(
                '{0} decided not to repair their {1}. '.format(
                entity.name, entity.property.occupancy.lower())
                )
            return

        yield env.process(assessment_program.process(entity.property, entity))
        yield env.process(permit_program.process(entity.property, entity))
        yield env.process(rebuild_program.process(entity.property, entity))

__A custom master process for OwnerHouseholds (owner occupiers). Don't do anything if no damage suffered. If residence damage is "Complete", abandon home and look to buy a different one. Otherwise look for financial assistance for repairs. If money for repairs can't be found (patience runs out), look for a new home. If home search patience runs out, simply stop.__

In [ ]:
def owner_process(env, inspection_program, insurance_program, fema_program, loan_program, 
                 assessment_program, permit_program, rebuild_program, forsale_stock, entity):
    
    money_patience = 1000  # days until give up the search for rebuild money
    home_patience = 15000  # days until give up the search for a new home
    
    yield env.process(inspection_program.process(entity.property, entity))
    
    # Specify the event sequence for households from the time of the hazard through the decisions to relocate 
    # or rebuild
    if entity.property.damage_state != 'None':

        # If home is completely damaged, search for a new home to purchase.
        if entity.property.damage_state == 'Complete':

            yield env.process(entity.replace_home(home_patience, forsale_stock)) 

            if entity.gave_up_home_search == None:
                yield env.process(entity.occupy(duration_prob_dist = scalar_dist))

            return
        
        # Search for financial assistance
        yield env.process(insurance_ia_loan.policy(insurance_prog, fema_prog,
                                                   loan_prog, entity, money_patience))
        
        if entity.gave_up_funding_search != None:
            return

        # If not enough money to repair home, search for a new home to purchase.
        if entity.money_to_rebuild < entity.property.damage_value:

            yield env.process(entity.replace_home(home_patience, forsale_stock)) 
            
            if entity.gave_up_home_search == None:
                yield env.process(entity.occupy(duration_prob_dist = scalar_dist))
            
            return
                
        elif entity.money_to_rebuild >= entity.property.damage_value:
            
            yield env.process(assessment_program.process(entity.property, entity))
            yield env.process(permit_program.process(entity.property, entity))
            yield env.process(rebuild_program.process(entity.property, entity))    
            yield env.process(entity.occupy(duration_prob_dist = scalar_dist))     

__A custom master process for RenterHouseholds. For the most part it simply initiates a process for their landlords. If they are evicted by their landlords, the renter will look for a new home. If home search patience runs out, simply stop. Otherwise, occupy home after landlord repairs it.__

In [ ]:
def renter_process(inspection_program, insurance_program, fema_program, loan_program, 
                 assessment_program, permit_program, rebuild_program, forrent_stock, entity):
        
    yield env.process(landlord_process(env, inspection_program, insurance_program, fema_program, 
                            loan_program, assessment_program, permit_program, rebuild_program, entity.landlord))

    if entity.residence != None:
        type(entity.residence)
        yield env.process(entity.occupy(duration_prob_dist = scalar_dist))
    else:
        search_patience = 550  # days until give up the search for a new home
        
        yield env.process(entity.replace_home(search_patience, forrent_stock))
        
        if not entity.gave_up_home_search:
                yield env.process(entity.occupy(duration_prob_dist = scalar_dist))

In [ ]:
forsale_process(forsale_stock):
    repair_fraction = 1.0
    repair_start = 0

    # start_delayed(env, repair_stock.policy(forsale_stock, repair_fraction), repair_start)
    yield repair_stock.policy(forsale_stock, repair_fraction, repair_start)

__Initiate the master process for each owner to be modeled in the simulation.__

In [ ]:
#inspect.getfullargspec(owner_process).args # Useful to determine what arguments are required for the process.

for i in range(len(owners)):
    env.process(owner_process(env, inspection_prog, insurance_prog, fema_prog, loan_prog, 
                                    assessment_prog, permit_prog, rebuild_prog, forsale_stock, owners[i]))

__Initiate the master process for each renter to be modeled in the simulation.__

In [ ]:
# #inspect.getfullargspec(renter_process).args # Useful to determine what arguments are required for the process.

# for i in range(len(renters)):
#     env.process(renter_process(inspection_prog, insurance_prog, fema_prog, loan_prog, 
#                                     assessment_prog, permit_prog, rebuild_prog, forrent_stock, renters[i]))

__Initiate the master process for each seller to be modeled in the simulation.__

In [ ]:
#inspect.getfullargspec(renter_process).args # Useful to determine what arguments are required for the process.


env.process(forsale_process(forsale_stock)

In [ ]:
# #inspect.getfullargspec(renter_process).args # Useful to determine what arguments are required for the process.

# for i in range(len(sellers)):
#     env.process(seller_process(inspection_prog, insurance_prog, fema_prog, loan_prog, 
#                                     assessment_prog, permit_prog, rebuild_prog, forsale_stock, sellers[i]))


__***CURRENTLY BROKEN*** Schedule an event that randomly fixes moderately or completely damaged homes in the vacant housing stock with probability = fix_probability__

In [ ]:
owners[3].property.damage_state

In [ ]:
for x in forsale_stock.items:
    print(x, x.damage_state)

In [ ]:
# repair_fraction = 1.0
# filter_fxn = lambda getBuilding: getBuilding.damage_state != 'None'

# for building in forsale_stock.items:
#     env.process(inspection_prog.process(building))

# dummy_entity_dict = {'Name': 'Rebuild Entity', 'Savings': float('inf'), 'Owner Insurance': 1.0}
# dummy_entity = Owner(env, dummy_entity_dict['Name'], dummy_entity_dict)

# random.seed(15)

# def filter_stock(stock, filter_fxn):
#     filtered_buildings = []  # Empty list to temporarily place FilterStore objects.
#     # Remove all structures from the FilterStore; put in a list for processing.
#     while forsale_stock.items:
#         get_building = yield stock.get(filter_fxn)
#         filtered_buildings.append(get_building)
    
# damaged_buildings = filter_stock(forsale_stock, filter_fxn)

# # Timeout until repair stock policy start time
# # yield self.env.timeout(start_time)

# num_fixed = 0  # Counter

# # Iterate through buildings, do processing, 
# for building in damaged_buildings:

#     # Compare uniform random to prob to estimate percentage to fix.
#     # Then yield assessment, permit, and repair processes
#     # Put back into the FilterStore
#     if random.uniform(0, 1.0) <= repair_fraction:
#         print(env.now)
#         env.process(assessment_prog.process(building, dummy_entity))
#         print(env.now)
#         env.process(permit_prog.process(building, dummy_entity))
#         print(env.now)
#         env.process(rebuild_prog.process(building, dummy_entity))
#         print(env.now)

# #         building.damage_state = 'None'
# #         building.damage_value = 0.0
# #         building_stock.put(building)
#         num_fixed += 1
#     else:
#         # Put back in FilterStore if not chosen to be fixed.
#         building_stock.put(building)

# print('{0} homes in the vacant building stock were repaired on day {1}.'.format(num_fixed, env.now))

__***CURRENTLY BROKEN*** Do inspections on all of the vacant rental stock.__
__Schedule an event that randomly fixes moderately or completely damaged homes in the vacant rental stock with probability = fix_probability__

In [ ]:
# for rental in forrent_stock.items:
#     env.process(inspection_prog.process(rental))
    
# fix_probability = 1.0
# fix_schedule = 100

# start_delayed(env, rebuild_stock_prog.process(forrent_stock, fix_probability), fix_schedule)

## Run the simulation

In [ ]:
env.run()

# Inspect Simulation Outputs

## OwnerHousehold summary statistics

In [ ]:
num_damaged = 0
num_rebuilt = 0
num_gave_up_money_search = 0
num_relocated = 0
num_homesearch = 0
num_gave_up_home_search = 0

for household in owners:
    if household.residence.damage_state != None: num_damaged += 1
    if household.repair_get != None: num_rebuilt += 1
    if household.gave_up_funding_search: num_gave_up_money_search += 1
    if household.home_search_start != None: num_homesearch += 1
    if household.home_search_stop != None: num_relocated += 1
    if household.gave_up_home_search: num_gave_up_home_search += 1
        
print('{0} out of {1} owners suffered damage to their homes.\n'.format(num_damaged, len(owners)),
      '{0} out of {1} owners rebuilt or repaired their damaged home.\n'.format(num_rebuilt, len(owners)),
        '{0} out of {1} owners gave up searching for money.\n'.format(num_gave_up_money_search, len(owners)),
      '{0} out of {1} owners searched for a new home.\n'.format(num_homesearch, len(owners)),
        '{0} out of {1} owners bought a new home.\n'.format(num_relocated, len(owners)),
        '{0} out of {1} owners gave up searching for a home.'.format(num_gave_up_home_search, len(owners))
      )

## Print OwnerHousehold stories

In [ ]:
owners[0].story

In [ ]:
owners[1].story

In [ ]:
owners[2].story

In [ ]:
owners[3].story

In [ ]:
owners[3].property.damage_value

In [ ]:
sellers[1].home_get

## RenterHousehold summary statistics

In [ ]:
num_damaged = 0
num_rebuilt = 0
num_relocated = 0
num_displaced = 0
num_gave_up_funding_search = 0
num_gave_up_home_search = 0

for renter in renters:

    if renter.landlord.property.damage_state != None: num_damaged += 1
    if renter.landlord.repair_get != None: num_rebuilt += 1
    if renter.landlord.gave_up_funding_search != None: num_gave_up_funding_search += 1
    if not renter.residence: num_displaced += 1
    if renter.gave_up_home_search: num_displaced += 1
        
print('{0} out of {1} renters\' homes suffered damage.\n'.format(num_damaged, len(renters)),
      '{0} out of {1} renters\' damaged home was rebuilt or repaired.\n'.format(num_rebuilt, len(renters)),
      '{0} out of {1} renters\' were displaced.\n'.format(num_displaced, len(renters)),
      '{0} landlords gave up searching for repair money'.format(num_gave_up_funding_search)
      )

## Print RenterHousehold stories

In [ ]:
renters[0].story + renters[0].landlord.story

In [ ]:
renters[1].story + renters[1].landlord.story

In [ ]:
renters[2].story + renters[2].landlord.story

In [ ]:
renters[3].story + renters[3].landlord.story

__*** This works but is a bandaid for saving simulation outputs for external visualization or stats. *** Create output file for visualizing__

In [ ]:
a = list(vars(owners[3]).keys()) #gets all potential column names
df = pd.DataFrame(columns=a)
iters = 0
att_itter = 0
new_column={}
log = []
for i in owners: #loop through all entities
#     i.latitude = i.owner["Latitude"] #extracting lat and long from the residence object
#     i.longitude = i.owner["Longitude"]
    for att in a: #loop through the attributes in our list of column names we want
        try:
            new_column[att] = i.__getattribute__(att) #set the b dictionary
            #mydata[att]= i.__getattribute__(att)
            
        except ValueError:
            new_column[att] = np.nan
        except AttributeError as e:
            new_column[att] = np.nan
            log.append("Household {0} had an attr error, {1}".format(i.name, e))
        finally:
            att_itter += 1
    mydata=pd.DataFrame([new_column]) #this turns our newly made column into a database where it can be combined with the df

    df = df.append(mydata, ignore_index=True)

    iters += 1

# output_path = "../outputs/output_df.csv"
# df.to_csv(output_path)
df.head()

In [ ]:
event_list=[]


for i in df.columns:
    if "get" in i or "put" in i or "stop" in i or "start" in i or "name" in i or "gave" in i:
        event_list.append(i)

event_df = df[event_list]
event_df = event_df.set_index('name')
event_df

__Example of how to visualize individual entities.__

In [ ]:
name = "Alfred"

name_row = df[df['name']==name][['inspection_put',
 'inspection_get',
 'assistance_put',
 'assistance_get',
 'assessment_put',
 'assessment_get',
 'permit_put',
 'permit_get',
 'home_put',
 'home_get']]

%matplotlib inline
plt.figure(figsize=(10,10))
sns.set_style(style="whitegrid")
sns.set(font_scale=2)
ax = sns.stripplot(name_row.ix[0], name_row.columns, jitter=True, size = 15, linewidth=1)
ax.set(xlabel="Days After Event", ylabel="Housing Recovery Events for {0}".format(name))